<div style='background-color: #87ceeb;
    border: 0.5em solid black;
    border-radius: 0.5em;
    padding: 1em;'>
    <h2>Activité</h2>
    <h1>Carte de vigilance météorologique</h1>
</div>

Les cartes de vigilance de Météo France permettent, depuis 2001, d'alerter la population avant la survenue de divers phénomènes tels que les vents violents, les fortes précipitations, les orages, les avalanches, etc. Elles ont été imaginées à la suite des deux tempêtes qui ont traversé la France les 26 et 27 décembre 1999 et ont tué 92 personnes.

Si elles avaient existé à l'époque, voici quelles auraient été les cartes de vigilance des 26 et 27 décembre 1999 :

<img src='https://ntoulzac.github.io/Cours-NSI-Terminale/api/images/vigilance_1999-12-26.svg' width="49%" style="display:inline-block;">
<img src='https://ntoulzac.github.io/Cours-NSI-Terminale/api/images/vigilance_1999-12-27.svg' width="49%" style="display:inline-block;">

*OpenDataSoft* met à disposition une API permettant de connaître, pour chaque département de l'Hexagone, la couleur associée à chaque risque (orages, canicule, _etc._).

Pour obtenir au format JSON l'ensemble des informations disponibles concernant le risque de canicule, par exemple, il suffit d'interroger l'API en se rendant à l'adresse :

[https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/weatherref-france-vigilance-meteo-departement/records?where=echeance:"J"%20and%20phenomenon:"orages"&limit=100&offset=0](https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/weatherref-france-vigilance-meteo-departement/records?where=echeance:"J"%20and%20phenomenon:"orages"&limit=100&offset=0)

**(1)** Après avoir cliqué sur le lien permettant d'interroger l'API et avoir affiché les données renvoyées par l'API sous forme de dictionnaire, déterminer :
- le nombre de clés que possède le dictionnaire,
- quel est le type de la valeur associée à la clé `results`, combien d'éléments elle contient et de quel type sont ces éléments.

On a isolé une partie des données concernant un département :

```json
{"domain_id": "94",
 "echeance": "J",
 "phenomenon_id": 6,
 "phenomenon": "canicule",
 "color_id": 1,
 "color": "vert",
 ...}
```

**(2)** Si on suppose que ces données sont stockées dans un dictionnaire appelé `dico`, déterminer quelles expressions
permettent d'obtenir :
- le numéro du département,
- le risque identifié,
- la couleur correspondant au risque identifié dans le département.

**(3)** Écrire la spécification de la fonction `interroger_API_vigilance_meteo`.

In [1]:
import requests

In [27]:
COULEURS = {'vert': '#008000', 'jaune': '#ffff00', 'orange': '#ff5e13', 'rouge': '#ff0000'}

def interroger_API_vigilance_meteo(risque):
    """
    Détermine, pour chaque département, sa couleur sur la carte de vigilance de Météo France pour un risque particulier.
    - Entrée : risque (chaîne de caractères)
    - Sortie : donnees (dictionnaire dont les clés sont les numéros des départements et la valeur associée
                     est le code RGB correspondant à la couleur du département)
    """
    donnees = {}
    for decalage in range(2):
        url = f"https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/weatherref-france-vigilance-meteo-departement/records?where=echeance:'J'%20and%20phenomenon:'{risque}'&limit=100&offset={100*decalage}"
        reponse = requests.get(url)
        reponse = reponse.json()
        for dico in reponse['results']:
            donnees[dico['domain_id']] = COULEURS[dico['color']]
    return donnees

La cellule suivante met à notre disposition une procédure `carte_departements` qui permet d'afficher une carte des départements de l'Hexagone en couleurs et, éventuellement, de l'enregistrer au format SVG. La procédure prend un ou deux paramètres d'entrée :
- un dictionnaire qui associe à chaque numéro de département un code couleur RGB,
- une chaîne de caractères optionnelle qui est le nom de fichier SVG dans lequel enregistrer la carte.

In [28]:
from carte_france import carte_departements

**(4)** Écrire des lignes de code permettant d'enregistrer dans un fichier `vigilance.svg` la carte de vigilance de Météo France pour le risque d'orages et de l'afficher dans le carnet *Jupyter*.

In [29]:
dictionnaire_couleurs = interroger_API_vigilance_meteo("orages")
carte_departements(dictionnaire_couleurs, "vigilance.svg")

<div style='background-color: #87ceeb;
    border-radius: 0.5em;
    padding: 1em;'>
    <h2>Pour aller plus loin...</h2>
</div>

L'API peut être interrogée sur les six risques suivants : `"canicule"`, `"crues"`, `"neige / verglas"`, `"orages"`, `"pluie"` et `"vent"`.

**(5)** Recopier et modifier la définition de la fonction `interroger_API_vigilance_meteo` de sorte que :
- la fonction ne prenne pas de paramètre d'entrée,
- l'API soit interrogée sur l'ensemble des six risques,
- soit retenue, pour chaque département, la couleur la plus "pessimiste" : par exemple, un département vert pour le risque crues mais orange pour le risque pluie devra apparaître en orange.

In [24]:
def interroger_API_vigilance_meteo():
    """
    Détermine, pour chaque département, sa couleur sur la carte de vigilance de Météo France.
    - Sortie : donnees (dictionnaire dont les clés sont les numéros des départements et la valeur associée
                     est le code RGB correspondant à la couleur du département)
    """
    TAB = [None, "vert", "jaune", "orange", "rouge"]
    donnees = {} 
    for risque in ["canicule", "crues", "neige / verglas", "orages", "pluie", "vent"]:
        for decalage in range(2):
            url = f"https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/weatherref-france-vigilance-meteo-departement/records?where=echeance:'J'%20and%20phenomenon:'{risque}'&limit=100&offset={100*decalage}"
            reponse = requests.get(url)
            reponse = reponse.json()
            for dico in reponse['results']:
                if dico['domain_id'] not in donnees or dico['color_id'] > donnees[dico['domain_id']]:
                    donnees[dico['domain_id']] = dico['color_id']
    for dep in donnees:
        donnees[dep] = COULEURS[TAB[donnees[dep]]]
    return donnees

In [25]:
dictionnaire_couleurs = interroger_API_vigilance_meteo()
carte_departements(dictionnaire_couleurs, "vigilance.svg")